In [1]:
# Importing required libraries
from selenium.common.exceptions import ElementNotInteractableException, NoSuchElementException
from selenium import webdriver
import pandas as pd
import time
import re

In [15]:
# Starting the browser
driver = webdriver.Edge("C:\\edgedriver_win64\\msedgedriver.exe")

In [16]:
# Opening the website
driver.get("https://www.monsterindia.com/")

In [17]:
# Entering "data science" in the search bar
search = driver.find_element_by_id('SE_home_autocomplete')
submit = driver.find_element_by_xpath('//*[@id="searchForm"]/div/div[2]/input')
search.send_keys("data science")
# clicking submit
submit.click()

## Extracting information from the filters tab

In [11]:
# Finding filter section on page
filters = driver.find_element_by_class_name('scroll-filter')

In [12]:
# Extracting filter list from the filters section
filter_list = filters.find_elements_by_class_name('accordion-panel.srp-filter-accordian.row')

In [13]:
filter_data = dict() # Dictionary to store filters data

# Extracting filter data from the list
for fil in filter_list:
    # Getting name of filter
    name = fil.find_element_by_css_selector('label h3').text
    # Maximizing filter list
    maximize = fil.find_element_by_class_name('icon-rgt.mqfi-plus')
    maximize.click()
    # Extracting filters options list
    ul = fil.find_element_by_css_selector('div ul')
    labels = ul.find_elements_by_class_name('checkboxLabel')
    label_list = [] # List to store the options of a filter
    for i in labels:
        label_list.append(i.text)
    # Minimizing filter list
    minimize = fil.find_element_by_class_name('icon-rgt.mqfi-minus')
    minimize.click()
    # Adding filter info to dictionary
    filter_data[name] = label_list
print(filter_data)

{'Function': ['IT\n(18227)', 'IT/Software Development - Systems/EDP/MIS\n(3884)', 'IT/Software Development - System Programming/Middleware\n(3491)', 'Manufacturing/Engineering/R&D\n(1584)', 'Engineering - Mechanical/Automotive/Industrial\n(1310)'], 'Role': ['Software Engineer/Programmer\n(15753)', 'Team Leader/Technical Leader\n(737)', 'Software Developer\n(448)', 'Application Engineer\n(378)', 'Correspondent/Reporter\n(368)'], 'Experience (In Years)': ['0 - 1 Years\n(5698)', '1 - 2 Years\n(8623)', '2 - 5 Years\n(17316)', '5 - 7 Years\n(18238)', '7 - 10 Years\n(15790)'], 'City': ['Bengaluru / Bangalore\n(10818)', 'Hyderabad\n(3197)', 'Pune\n(2102)', 'Chennai\n(1849)', 'Hyderabad / Secunderabad\n(1814)'], 'Salary (INR)': ['0 - 1Lakhs\n(159)', '1Lakhs - 2Lakhs\n(169)', '2Lakhs - 5Lakhs\n(378)', '5Lakhs - 10Lakhs\n(644)', '10Lakhs - 20Lakhs\n(590)'], 'Great Place To Work': ['Show Only GPTW Jobs\n(319)'], 'Qualification': ['Bachelor Of Technology (B.Tech/B.E)\n(322)', 'Aviation\n(198)', 'M

In [14]:
clean_filter_data = {} # Dictionary to store cleaned filter data
# Cleaning filter data
for l in filter_data:
    items = [] # List to store cleaned filter items
    for item in filter_data[l]:
        split_data = item.split('\n') # Splitting item into option name and count of records
        split_data[1] = int(split_data[1][1:-2]) # Removing brackets in count of records and converting to integer
        items.append(split_data) # Appending to list
    clean_filter_data[l] = items # Adding to new dictionary
print(clean_filter_data)

{'Function': [['IT', 1822], ['IT/Software Development - Systems/EDP/MIS', 388], ['IT/Software Development - System Programming/Middleware', 349], ['Manufacturing/Engineering/R&D', 158], ['Engineering - Mechanical/Automotive/Industrial', 131]], 'Role': [['Software Engineer/Programmer', 1575], ['Team Leader/Technical Leader', 73], ['Software Developer', 44], ['Application Engineer', 37], ['Correspondent/Reporter', 36]], 'Experience (In Years)': [['0 - 1 Years', 569], ['1 - 2 Years', 862], ['2 - 5 Years', 1731], ['5 - 7 Years', 1823], ['7 - 10 Years', 1579]], 'City': [['Bengaluru / Bangalore', 1081], ['Hyderabad', 319], ['Pune', 210], ['Chennai', 184], ['Hyderabad / Secunderabad', 181]], 'Salary (INR)': [['0 - 1Lakhs', 15], ['1Lakhs - 2Lakhs', 16], ['2Lakhs - 5Lakhs', 37], ['5Lakhs - 10Lakhs', 64], ['10Lakhs - 20Lakhs', 59]], 'Great Place To Work': [['Show Only GPTW Jobs', 31]], 'Qualification': [['Bachelor Of Technology (B.Tech/B.E)', 32], ['Aviation', 19], ['Master in Computer Applicati

## Extracting job posts

In [143]:
#driver.refresh()
job_posts = []

#for i in range(0,50):
while(True):
    
    # Without Filters
    jobs = driver.find_element_by_xpath('//*[@id="srp-right-part"]/div/div[1]/div/div/div[2]/div')
    
    # When Filters are applied
    #jobs = driver.find_element_by_xpath('//*[@id="srp-right-part"]/div/div[1]/div/div/div[3]/div')
    
    # Referring to the section with job posts
    posts = jobs.find_elements_by_class_name('card-panel.apply-panel.job-apply-card')

    # Loading all posts by scrolling
    for i in range(0,len(posts),4):
        driver.execute_script("arguments[0].scrollIntoView();",posts[i])
        time.sleep(3)

    # Getting final list of posts
    posts = jobs.find_elements_by_class_name('card-panel.apply-panel.job-apply-card')
    print("No of posts:",len(posts))
    
    # Extracting job details from each job post
    for job in posts:
        
        # Title
        title_block = job.find_element_by_class_name('job-tittle')
        title = title_block.find_element_by_css_selector('h3 a').text
        
        try:
            company = title_block.find_element_by_css_selector('span.company-name a').text # Company Name
        except:
            pass
        try:
            location = title_block.find_element_by_css_selector('div span small').text # Job Location
        except:
            pass
        try:
            experience = title_block.find_element_by_css_selector('div.exp span small').text # Experience Needed
        except:
            pass
        try:
            package = title_block.find_element_by_css_selector('div.package span small').text # Salary Estimate
        except:
            pass

        # Description
        try:
            description = job.find_element_by_class_name('job-descrip').text # Job Description
        except:
            pass

        # Fetching list of Skills needed
        try:
            skills = []
            skills_block = job.find_element_by_class_name('descrip-skills')
        except NoSuchElementException:
            pass
        else:
            skills_list = skills_block.find_elements_by_tag_name('a')
            for skill in skills_list:
                skills.append(skill.text)
            
        post = [title,company,location,experience,package,description,skills]
        job_posts.append(post)

    # Going to next page
    try:
        next_page = driver.find_element_by_class_name('btn-next')
        time.sleep(3)
        next_page.click()
        time.sleep(3)
    except ElementNotInteractableException:
        break

len(job_posts)

No of posts: 10
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of posts: 25
No of po

WebDriverException: Message: disconnected: received Inspector.detached event
  (Session info: MicrosoftEdge=98.0.1108.50)


In [144]:
jobs_df = pd.DataFrame(job_posts, columns = ['Title','Company','Location','Experience','Package','Description','Skills'])
jobs_df

,Title,Company,Location,Experience,Package,Description,Skills
0,Data Science Manager,Amazon,Bengaluru / Bangalore,10-13 Years,Not Specified,Job Description : Hundreds of millions of cust...,[Data Science Manager]
1,Data Science Engineer,FactSet Philippines Inc,India,Not Specified,Not Specified,VALUES THAT DEFINE OUR CULTURE We are unified ...,[]
2,Manager Data Science,Reliance Jio Infocomm Limited,Mumbai,5-8 Years,Not Specified,Job Description : * Job Responsibilities : 1. ...,[]
3,Manager Data Science,Reliance Jio Infocomm Limited,Bengaluru / Bangalore,5-8 Years,Not Specified,Job Description : * Job Responsibilities : 1. ...,[]
4,Manager Data Science,Reliance Jio Infocomm Limited,Bengaluru / Bangalore,5-8 Years,Not Specified,Job Description : * Job Responsibilities : 1. ...,[]
...,...,...,...,...,...,...,...
5476,Transformation BD,Amazon,Gurgaon / Gurugram,7-10 Years,Not Specified,Job Description : Preferred locations: Bangalo...,[Speciality BD Executive]
5477,Virtual Agents Application Developer,Accenture,"Bengaluru / Bangalore, India",12-15 Years,Not Specified,About Accenture: Accenture is a global profess...,[Software Engineering]
5478,Sr Advanced Project Engr,Honeywell,"India, Bengaluru / Bangalore",6-9 Years,Not Specified,"Join a team recognized for leadership, innovat...","[Safe ,, leaders ,, Drives ,, project manageme..."
5479,MTS 1 Full Stack Engineer,PayPal,"Bengaluru / Bangalore, India",8-11 Years,Not Specified,Job Description : Job Description At PayPal (N...,[MTS 1 Full Stack Engineer]


In [145]:
# Cleaning Job Posts Skills Data
skills = jobs_df['Skills']
clean_skills = [] # List to store clean data
for li in skills:
    # Replacing unnecessary strings from the list
    li = list(map(lambda st: str.replace(st, ' ,',''), li))
    clean_skills.append(li)
jobs_df['Skills'] = clean_skills # Updating the dataframe

In [146]:
jobs_df.iloc[:,:-1].nunique()

Title          2188
Company         135
Location         96
Experience       96
Package          48
Description    2689
dtype: int64

In [147]:
jobs_df['Experience'] = jobs_df['Experience'].apply(lambda st: str.replace(st, ' Years',''))

In [148]:
jobs_df['Experience'] = jobs_df['Experience'].apply(lambda st: str.split(st, '-'))
jobs_df['Experience']

0              [10, 13]
1       [Not Specified]
2                [5, 8]
3                [5, 8]
4                [5, 8]
             ...       
5476            [7, 10]
5477           [12, 15]
5478             [6, 9]
5479            [8, 11]
5480           [10, 13]
Name: Experience, Length: 5481, dtype: object

In [149]:
jobs_df['Min_Experience'] = jobs_df['Experience'].apply(lambda st: st[0] if len(st) == 2 else None)
jobs_df['Max_Experience'] = jobs_df['Experience'].apply(lambda st: st[1] if len(st) == 2 else None)
jobs_df.drop('Experience',axis=1, inplace=True)

In [150]:
jobs_df

,Title,Company,Location,Package,Description,Skills,Min_Experience,Max_Experience
0,Data Science Manager,Amazon,Bengaluru / Bangalore,Not Specified,Job Description : Hundreds of millions of cust...,[Data Science Manager],10,13
1,Data Science Engineer,FactSet Philippines Inc,India,Not Specified,VALUES THAT DEFINE OUR CULTURE We are unified ...,[],None,None
2,Manager Data Science,Reliance Jio Infocomm Limited,Mumbai,Not Specified,Job Description : * Job Responsibilities : 1. ...,[],5,8
3,Manager Data Science,Reliance Jio Infocomm Limited,Bengaluru / Bangalore,Not Specified,Job Description : * Job Responsibilities : 1. ...,[],5,8
4,Manager Data Science,Reliance Jio Infocomm Limited,Bengaluru / Bangalore,Not Specified,Job Description : * Job Responsibilities : 1. ...,[],5,8
...,...,...,...,...,...,...,...,...
5476,Transformation BD,Amazon,Gurgaon / Gurugram,Not Specified,Job Description : Preferred locations: Bangalo...,[Speciality BD Executive],7,10
5477,Virtual Agents Application Developer,Accenture,"Bengaluru / Bangalore, India",Not Specified,About Accenture: Accenture is a global profess...,[Software Engineering],12,15
5478,Sr Advanced Project Engr,Honeywell,"India, Bengaluru / Bangalore",Not Specified,"Join a team recognized for leadership, innovat...","[Safe, leaders, Drives, project management, ve...",6,9
5479,MTS 1 Full Stack Engineer,PayPal,"Bengaluru / Bangalore, India",Not Specified,Job Description : Job Description At PayPal (N...,[MTS 1 Full Stack Engineer],8,11


In [151]:
jobs_df.to_csv('Monster_DS_Job_Posts.csv')